In [3]:
import glob
import os
import scipy
import math
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import statsmodels.stats.multitest as mt
from pathlib import Path
from scipy.stats import ttest_rel
from scipy.stats import shapiro

In [4]:
mse_list = []
mae_list = []
representation_name_list = []
for path in Path("/media/DATA/serbulent/DATA/Thesis/ReviewPaper/results/revision-1/affinity_prediction/")\
.glob("Affinity_prediction_skempiv1_*_detail.csv"):
    #print(path)
    representation_name_list.append(str(path).split("skempiv1_")[1].split("_detail.csv")[0])
    mse_list.append(pd.read_csv(path)['val_mse_errors'])
    mae_list.append(pd.read_csv(path)['val_mae_errors'])
    
df_mse = pd.concat(mse_list, axis=1)
df_mse.columns = representation_name_list
#Sorting columns by their mean value
df_mse = df_mse.reindex(df_mse.mean().sort_values().index, axis=1)

df_mae = pd.concat(mae_list, axis=1)
df_mae.columns = representation_name_list
#Sorting columns by their mean value
df_mae = df_mae.reindex(df_mae.mean().sort_values().index, axis=1)

In [5]:
df_mse

ALBERT     ESMB1    SEQVEC  BERT-PFAM  BERT-BFD        T5     XLNET  \
0  0.363818  0.450163  0.485027   0.486842  0.444817  0.542026  0.575570   
1  0.414565  0.493169  0.589129   0.609138  0.611387  0.537978  0.682961   
2  0.422572  0.434206  0.538995   0.532121  0.926363  0.527603  0.569832   
3  0.418076  0.621966  0.600896   0.557599  0.566967  0.587362  0.712379   
4  0.376907  0.476562  0.511827   0.550132  0.486599  0.531136  0.587876   
5  0.385243  0.488588  0.482960   0.522924  0.529628  0.504556  0.512663   
6  0.533704  0.435311  0.587326   0.583105  0.671858  1.043043  0.695626   
7  0.515289  0.549829  0.586660   0.639272  0.555272  0.518099  0.655713   
8  0.316215  0.414722  0.498666   0.588213  0.463734  0.657308  0.520614   
9  0.436316  0.463862  0.449098   0.591227  0.479161  0.585781  0.554477   

   CPC-PROT    UNIREP  MSA_Transformer     K-SEP   PROTVEC  LEARNED-VEC  \
0  0.613292  0.660293         0.834127  0.819674  0.945078     1.016695   
1  0.738369  0.762166         0.935961  1.074306  1.107979     1.161348   
2  0.854581  0.686010         0.918776  1.074676  1.184138     1.279863   
3  0.808556  0.732935         0.919206  1.072209  1.305883     1.227308   
4  0.663005  0.683952         0.888975  0.846457  1.094767     1.171091   
5  0.621654  0.658096         0.743666  0.989920  1.077726     1.047167   
6  0.803881  0.712041         0.965069  0.856228  1.134837     1.250117   
7  0.826819  0.888054         0.967583  1.137318  1.158530     1.253739   
8  0.664550  0.761072         0.888210  0.981106  1.154163     1.133077   
9  0.704288  0.776349         1.018583  0.853148  1.177525     1.307355   

      APAAC       AAC      PFAM  
0  1.756543  1.683668  2.108073  
1  1.877006  1.948253  2.369755  
2  1.756783  1.944204  2.273111  
3  1.938557  1.930600  2.424406  
4  1.714857  1.696047  2.281235  
5  1.498729  1.648721  2.058992  
6  1.738988  1.854286  2.254950  
7  1.984052  1.839930  2.227672  
8  1.810791  2.013785  2.342703  
9  1.824198  1.918340  2.237945

In [6]:
df_mae

ALBERT     ESMB1  BERT-BFD    SEQVEC        T5  BERT-PFAM     XLNET  \
0  4.543807  4.894548  4.890892  5.183610  5.409785   5.208129  5.712806   
1  4.620814  5.203315  5.725314  5.630844  5.470758   5.884786  6.116328   
2  4.636323  4.838902  5.331702  5.092170  5.239655   5.217586  5.498052   
3  4.629862  5.299021  5.401384  5.296863  5.719941   5.594848  6.026316   
4  4.461181  4.840316  4.978716  5.274798  5.323068   5.705067  5.722243   
5  4.421143  5.213292  5.043597  5.114270  5.116063   5.501968  5.331993   
6  4.601095  4.819886  5.207438  5.296176  5.682376   5.525344  5.824787   
7  4.921624  5.166240  5.471131  5.459542  5.473452   5.861534  5.876339   
8  4.130872  4.745919  4.908036  5.082051  5.595247   5.475668  5.408661   
9  4.761172  4.997554  5.159425  4.992459  5.589385   5.704265  5.483145   

   CPC-PROT    UNIREP  MSA_Transformer     K-SEP   PROTVEC  LEARNED-VEC  \
0  5.881050  6.210119         6.902550  6.839082  7.703737     7.483917   
1  6.514593  6.694845         7.333535  7.606413  8.023558     8.120023   
2  6.238775  6.144276         7.000387  7.254431  8.231402     8.065292   
3  6.503795  6.324278         7.313779  7.621369  8.616002     8.125022   
4  6.373199  6.402114         7.122091  6.747383  7.900280     8.318308   
5  5.808810  6.065039         6.524238  7.267059  7.859473     7.865724   
6  6.220070  6.079507         7.081517  6.772668  7.918496     8.248700   
7  6.900368  7.049279         7.516087  7.930164  8.399731     8.793236   
8  5.966784  6.301896         6.829892  7.055929  8.026435     7.715767   
9  6.201759  6.602532         7.326127  6.978653  8.210337     8.572173   

       APAAC        AAC       PFAM  
0  10.231082  10.228430  11.528571  
1  10.339209  11.031641  12.113383  
2   9.883432  10.884177  11.882772  
3  10.739856  10.757778  12.039008  
4   9.905500  10.321767  11.845835  
5   9.479030   9.540096  10.687141  
6  10.167630  10.564566  11.633480  
7  10.826505  10.717635  11.645657  
8  10.267877  10.981401  11.767190  
9  10.427031  10.903877  11.784157

In [7]:
def check_for_normality(go_pred_signinificance_score_df):
    for col in go_pred_signinificance_score_df:
        stat, p = shapiro(go_pred_signinificance_score_df[col])
        # interpret
        alpha = 0.05
        if p < alpha:
            print('Data does not drawn from a Normal distribution (reject H0) for ' + col)
            print('Statistics=%.3f, p=%.3f' % (stat, p))

In [8]:
check_for_normality(df_mae)

Data does not drawn from a Normal distribution (reject H0) for PFAM
Statistics=0.802, p=0.015


In [9]:
check_for_normality(df_mse)

Data does not drawn from a Normal distribution (reject H0) for BERT-BFD
Statistics=0.806, p=0.017
Data does not drawn from a Normal distribution (reject H0) for T5
Statistics=0.602, p=0.000


In [10]:
scipy.stats.wilcoxon(df_mse['ALBERT'], df_mse['PFAM'],zero_method='zsplit')

WilcoxonResult(statistic=0.0, pvalue=0.001953125)

In [11]:
scipy.stats.wilcoxon(df_mse['ALBERT'], df_mse['SEQVEC'],zero_method='zsplit')

WilcoxonResult(statistic=0.0, pvalue=0.001953125)

In [12]:
scipy.stats.wilcoxon(df_mae['ALBERT'], df_mae['PFAM'],zero_method='zsplit')

WilcoxonResult(statistic=0.0, pvalue=0.001953125)

In [13]:
scipy.stats.wilcoxon(df_mae['ALBERT'], df_mae['SEQVEC'],zero_method='zsplit')

WilcoxonResult(statistic=0.0, pvalue=0.001953125)

In [14]:
ttest_rel(df_mse['ALBERT'], df_mse['PFAM'])

Ttest_relResult(statistic=-45.26877170213428, pvalue=6.26464552949668e-12)

In [15]:
ttest_rel(df_mse['ALBERT'], df_mse['SEQVEC'])

Ttest_relResult(statistic=-6.334666834470753, pvalue=0.00013530681621252668)

In [16]:
list(df_mse['ALBERT'])

[0.3638181205766389,
 0.4145647246062643,
 0.4225723542059362,
 0.4180755689420302,
 0.3769070687465099,
 0.3852427891764156,
 0.5337044027341763,
 0.5152892577275208,
 0.3162150639244056,
 0.4363157506928524]

In [17]:
list(df_mse['SEQVEC'])

[0.4850268762927039,
 0.5891293710389106,
 0.5389950466321576,
 0.6008960241488368,
 0.5118265717285728,
 0.4829595185286985,
 0.5873263197580205,
 0.5866599413281838,
 0.4986661814715105,
 0.4490983245787546]

In [18]:
list(df_mse['PFAM'])

[2.1080725198514347,
 2.369754712761041,
 2.2731113271623857,
 2.4244058851458106,
 2.2812349641007827,
 2.058992245277642,
 2.2549504748938407,
 2.227671872078743,
 2.3427025287118317,
 2.237944953316925]

In [19]:
def nan_to_zero(x):
    if math.isnan(x):
        return 0
    else:
        return x

def calculate_q_vals(pred_score_table):
    tTest_pvalue_list = []
    for model1 in pred_score_table:
        for model2 in pred_score_table:
            if model1 != model2:
                t_result = scipy.stats.wilcoxon(pred_score_table[model1], pred_score_table[model2])
                tTest_pvalue_list.append(t_result.pvalue)
            else:
                tTest_pvalue_list.append(1.0)

    tTest_pvalue_list = [nan_to_zero(p_val) for p_val in tTest_pvalue_list]
    #multiple test correction using  Benjamini/Hochberg
    q_vals = mt.multipletests(tTest_pvalue_list,alpha=0.05,method='fdr_bh')[1]

    #convert q value list to dataframe
    model_number = len(pred_score_table.columns)
    chunks = [q_vals[x:x+model_number] for x in range(0, len(q_vals), model_number)]
    qval_df = pd.DataFrame(chunks,columns=pred_score_table.columns)

    qval_df.set_index(pred_score_table.columns, inplace=True)
    return qval_df

In [20]:
mse_qval_df = calculate_q_vals(df_mse)
mse_qval_df

ALBERT     ESMB1    SEQVEC  BERT-PFAM  BERT-BFD        T5  \
ALBERT           1.000000  0.033981  0.002632   0.002632  0.002632  0.002632   
ESMB1            0.033981  1.000000  0.033981   0.012500  0.058962  0.024752   
SEQVEC           0.002632  0.033981  1.000000   0.150901  0.834746  0.260965   
BERT-PFAM        0.002632  0.012500  0.150901   1.000000  0.760684  1.000000   
BERT-BFD         0.002632  0.058962  0.834746   0.760684  1.000000  0.543103   
T5               0.002632  0.024752  0.260965   1.000000  0.543103  1.000000   
XLNET            0.002632  0.002632  0.002632   0.098624  0.122727  0.480435   
CPC-PROT         0.002632  0.002632  0.002632   0.002632  0.005051  0.058962   
UNIREP           0.002632  0.002632  0.002632   0.002632  0.045673  0.076389   
MSA_Transformer  0.002632  0.002632  0.002632   0.002632  0.005051  0.005051   
K-SEP            0.002632  0.002632  0.002632   0.002632  0.002632  0.005051   
PROTVEC          0.002632  0.002632  0.002632   0.002632  0.002632  0.002632   
LEARNED-VEC      0.002632  0.002632  0.002632   0.002632  0.002632  0.002632   
APAAC            0.002632  0.002632  0.002632   0.002632  0.002632  0.002632   
AAC              0.002632  0.002632  0.002632   0.002632  0.002632  0.002632   
PFAM             0.002632  0.002632  0.002632   0.002632  0.002632  0.002632   

                    XLNET  CPC-PROT    UNIREP  MSA_Transformer     K-SEP  \
ALBERT           0.002632  0.002632  0.002632         0.002632  0.002632   
ESMB1            0.002632  0.002632  0.002632         0.002632  0.002632   
SEQVEC           0.002632  0.002632  0.002632         0.002632  0.002632   
BERT-PFAM        0.098624  0.002632  0.002632         0.002632  0.002632   
BERT-BFD         0.122727  0.005051  0.045673         0.005051  0.002632   
T5               0.480435  0.058962  0.076389         0.005051  0.005051   
XLNET            1.000000  0.002632  0.002632         0.002632  0.002632   
CPC-PROT         0.002632  1.000000  0.909664         0.002632  0.002632   
UNIREP           0.002632  0.909664  1.000000         0.002632  0.002632   
MSA_Transformer  0.002632  0.002632  0.002632         1.000000  0.260965   
K-SEP            0.002632  0.002632  0.002632         0.260965  1.000000   
PROTVEC          0.002632  0.002632  0.002632         0.002632  0.002632   
LEARNED-VEC      0.002632  0.002632  0.002632         0.002632  0.002632   
APAAC            0.002632  0.002632  0.002632         0.002632  0.002632   
AAC              0.002632  0.002632  0.002632         0.002632  0.002632   
PFAM             0.002632  0.002632  0.002632         0.002632  0.002632   

                  PROTVEC  LEARNED-VEC     APAAC       AAC      PFAM  
ALBERT           0.002632     0.002632  0.002632  0.002632  0.002632  
ESMB1            0.002632     0.002632  0.002632  0.002632  0.002632  
SEQVEC           0.002632     0.002632  0.002632  0.002632  0.002632  
BERT-PFAM        0.002632     0.002632  0.002632  0.002632  0.002632  
BERT-BFD         0.002632     0.002632  0.002632  0.002632  0.002632  
T5               0.002632     0.002632  0.002632  0.002632  0.002632  
XLNET            0.002632     0.002632  0.002632  0.002632  0.002632  
CPC-PROT         0.002632     0.002632  0.002632  0.002632  0.002632  
UNIREP           0.002632     0.002632  0.002632  0.002632  0.002632  
MSA_Transformer  0.002632     0.002632  0.002632  0.002632  0.002632  
K-SEP            0.002632     0.002632  0.002632  0.002632  0.002632  
PROTVEC          1.000000     0.076389  0.002632  0.002632  0.002632  
LEARNED-VEC      0.076389     1.000000  0.002632  0.002632  0.002632  
APAAC            0.002632     0.002632  1.000000  0.220982  0.002632  
AAC              0.002632     0.002632  0.220982  1.000000  0.002632  
PFAM             0.002632     0.002632  0.002632  0.002632  1.000000

In [21]:
mae_qval_df = calculate_q_vals(df_mae)
mae_qval_df

ALBERT     ESMB1  BERT-BFD    SEQVEC        T5  BERT-PFAM  \
ALBERT           1.000000  0.002358  0.002358  0.002358  0.002358   0.002358   
ESMB1            0.002358  1.000000  0.041667  0.031250  0.004630   0.002358   
BERT-BFD         0.002358  0.041667  1.000000  0.820833  0.041667   0.004630   
SEQVEC           0.002358  0.031250  0.820833  1.000000  0.022523   0.002358   
T5               0.002358  0.004630  0.041667  0.022523  1.000000   0.472222   
BERT-PFAM        0.002358  0.002358  0.004630  0.002358  0.472222   1.000000   
XLNET            0.002358  0.002358  0.002358  0.002358  0.016055   0.144397   
CPC-PROT         0.002358  0.002358  0.002358  0.002358  0.002358   0.002358   
UNIREP           0.002358  0.002358  0.002358  0.002358  0.002358   0.002358   
MSA_Transformer  0.002358  0.002358  0.002358  0.002358  0.002358   0.002358   
K-SEP            0.002358  0.002358  0.002358  0.002358  0.002358   0.002358   
PROTVEC          0.002358  0.002358  0.002358  0.002358  0.002358   0.002358   
LEARNED-VEC      0.002358  0.002358  0.002358  0.002358  0.002358   0.002358   
APAAC            0.002358  0.002358  0.002358  0.002358  0.002358   0.002358   
AAC              0.002358  0.002358  0.002358  0.002358  0.002358   0.002358   
PFAM             0.002358  0.002358  0.002358  0.002358  0.002358   0.002358   

                    XLNET  CPC-PROT    UNIREP  MSA_Transformer     K-SEP  \
ALBERT           0.002358  0.002358  0.002358         0.002358  0.002358   
ESMB1            0.002358  0.002358  0.002358         0.002358  0.002358   
BERT-BFD         0.002358  0.002358  0.002358         0.002358  0.002358   
SEQVEC           0.002358  0.002358  0.002358         0.002358  0.002358   
T5               0.016055  0.002358  0.002358         0.002358  0.002358   
BERT-PFAM        0.144397  0.002358  0.002358         0.002358  0.002358   
XLNET            1.000000  0.002358  0.002358         0.002358  0.002358   
CPC-PROT         0.002358  1.000000  0.093478         0.002358  0.002358   
UNIREP           0.002358  0.093478  1.000000         0.002358  0.002358   
MSA_Transformer  0.002358  0.002358  0.002358         1.000000  0.672269   
K-SEP            0.002358  0.002358  0.002358         0.672269  1.000000   
PROTVEC          0.002358  0.002358  0.002358         0.002358  0.002358   
LEARNED-VEC      0.002358  0.002358  0.002358         0.002358  0.002358   
APAAC            0.002358  0.002358  0.002358         0.002358  0.002358   
AAC              0.002358  0.002358  0.002358         0.002358  0.002358   
PFAM             0.002358  0.002358  0.002358         0.002358  0.002358   

                  PROTVEC  LEARNED-VEC     APAAC       AAC      PFAM  
ALBERT           0.002358     0.002358  0.002358  0.002358  0.002358  
ESMB1            0.002358     0.002358  0.002358  0.002358  0.002358  
BERT-BFD         0.002358     0.002358  0.002358  0.002358  0.002358  
SEQVEC           0.002358     0.002358  0.002358  0.002358  0.002358  
T5               0.002358     0.002358  0.002358  0.002358  0.002358  
BERT-PFAM        0.002358     0.002358  0.002358  0.002358  0.002358  
XLNET            0.002358     0.002358  0.002358  0.002358  0.002358  
CPC-PROT         0.002358     0.002358  0.002358  0.002358  0.002358  
UNIREP           0.002358     0.002358  0.002358  0.002358  0.002358  
MSA_Transformer  0.002358     0.002358  0.002358  0.002358  0.002358  
K-SEP            0.002358     0.002358  0.002358  0.002358  0.002358  
PROTVEC          1.000000     0.672269  0.002358  0.002358  0.002358  
LEARNED-VEC      0.672269     1.000000  0.002358  0.002358  0.002358  
APAAC            0.002358     0.002358  1.000000  0.022523  0.002358  
AAC              0.002358     0.002358  0.022523  1.000000  0.002358  
PFAM             0.002358     0.002358  0.002358  0.002358  1.000000

In [22]:
path = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/results/revision-1/"
mse_qval_df.to_csv(path + "affinity_table_mse_significance.csv")
mae_qval_df.to_csv(path + "affinity_table_mae_significance.csv")